In [1]:
#https://www.pyimagesearch.com/2016/10/03/bubble-sheet-multiple-choice-scanner-and-test-grader-using-omr-python-and-opencv/
# import the necessary packages
from imutils.perspective import four_point_transform
from imutils import contours
import numpy as np
import imutils
import cv2
 
# define the answer key which maps the question number
# to the correct answer
ANSWER_KEY = {0: 1, 1: 4, 2: 0, 3: 3, 4: 1}

In [2]:
# load the image, convert it to grayscale, blur it
# slightly, then find edges
image = cv2.imread('./omr_test_01.png')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (3, 3), 0)
edged = cv2.Canny(blurred, 75, 200)


In [43]:
cv2.imshow("edged", edged)
cv2.waitKey(0)

-1

Notice how the edges of the document are clearly defined, with all four vertices of the exam being present in the image.

Obtaining this silhouette of the document is extremely important in our next step as we will use it as a marker to apply a perspective transform to the exam, obtaining a top-down, birds-eye-view of the document:

In [7]:
# find contours in the edge map, then initialize
# the contour that corresponds to the document
cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
docCnt = None
 
# ensure that at least one contour was found
if len(cnts) > 0:
    # sort the contours according to their size in
    # descending order
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
 
    # loop over the sorted contours
    for c in cnts:
        # approximate the contour
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
 
        # if our approximated contour has four points,
        # then we can assume we have found the paper
        if len(approx) == 4:
            docCnt = approx
            break

In [79]:
for c in cnts:
    # compute the bounding box of the contour and then draw the
    # bounding box on both input images to represent where the two
    # images differ
    (x, y, w, h) = cv2.boundingRect(c)
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 2)

cv2.imshow("image", image)
cv2.waitKey(0)

-1

Now that we have the outline of our exam, we apply the cv2.findContours  function to find the lines that correspond to the exam itself.

We do this by sorting our contours by their area (from largest to smallest) on Line 37 (after making sure at least one contour was found on Line 34, of course). This implies that larger contours will be placed at the front of the list, while smaller contours will appear farther back in the list.

We make the assumption that our exam will be the main focal point of the image, and thus be larger than other objects in the image. This assumption allows us to “filter” our contours, simply by investigating their area and knowing that the contour that corresponds to the exam should be near the front of the list.

However, contour area and size is not enough — we should also check the number of vertices on the contour.

To do, this, we loop over each of our (sorted) contours on Line 40. For each of them, we approximate the contour, which in essence means we simplify the number of points in the contour, making it a “more basic” geometric shape. You can read more about contour approximation in this post on building a mobile document scanner.

On Line 47 we make a check to see if our approximated contour has four points, and if it does, we assume that we have found the exam.

Below I have included an example image that demonstrates the docCnt  variable being drawn on the original image:

In [24]:
# apply a four point perspective transform to both the
# original image and grayscale image to obtain a top-down
# birds eye view of the paper
paper = four_point_transform(image, docCnt.reshape(4, 2))
warped = four_point_transform(gray, docCnt.reshape(4, 2))

In [25]:
# apply Otsu's thresholding method to binarize the warped
# piece of paper
thresh = cv2.threshold(warped, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

In [73]:
cv2.imshow("thresh", thresh)
cv2.waitKey(0)

-1

In [87]:
# find contours in the thresholded image, then initialize
# the list of contours that correspond to questions
cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
questionCnts = []
 
# loop over the contours
for c in cnts:
    # compute the bounding box of the contour, then use the
    # bounding box to derive the aspect ratio
    (x, y, w, h) = cv2.boundingRect(c)
    ar = w / float(h)
 
    # in order to label the contour as a question, region
    # should be sufficiently wide, sufficiently tall, and
    # have an aspect ratio approximately equal to 1
    if w >= 20 and h >= 20 and ar >= 0.9 and ar <= 1.1:
        questionCnts.append(c)

Lines 64-67 handle finding contours on our thresh  binary image, followed by initializing questionCnts , a list of contours that correspond to the questions/bubbles on the exam.

To determine which regions of the image are bubbles, we first loop over each of the individual contours (Line 70).

For each of these contours, we compute the bounding box (Line 73), which also allows us to compute the aspect ratio, or more simply, the ratio of the width to the height (Line 74).

In order for a contour area to be considered a bubble, the region should:

Be sufficiently wide and tall (in this case, at least 20 pixels in both dimensions).
Have an aspect ratio that is approximately equal to 1.
As long as these checks hold, we can update our questionCnts  list and mark the region as a bubble.

Below I have included a screenshot that has drawn the output of questionCnts  on our image:

In [99]:
# loop over the contours
output = paper.copy()
for question in questionCnts:
    # compute the bounding box of the contour and then draw the
    # bounding box on both input images to represent where the two
    # images differ
    contours_poly = cv2.approxPolyDP(question, 3, True)
    
    center, radiu = cv2.minEnclosingCircle(contours_poly)
    
    
    cv2.circle(output, (int(center[0]), int(center[1])), int(radiu), (0, 0, 255), 2)


cv2.imshow("questions", output)
cv2.waitKey(0)

-1

Notice how only the question regions of the exam are highlighted and nothing else.

We can now move on to the “grading” portion of our OMR system:



In [102]:
# sort the question contours top-to-bottom, then initialize
# the total number of correct answers
questionCnts = contours.sort_contours(questionCnts, method="top-to-bottom")[0]
correct = 0

# each question has 5 possible answers, to loop over the
# question in batches of 5
for (q, i) in enumerate(np.arange(0, len(questionCnts), 5)):
    # sort the contours for the current question from
    # left to right, then initialize the index of the
    # bubbled answer
    cnts = contours.sort_contours(questionCnts[i:i + 5])[0]
    bubbled = None
    
    # loop over the sorted contours
    for (j, c) in enumerate(cnts):
        # construct a mask that reveals only the current
        # "bubble" for the question
        mask = np.zeros(thresh.shape, dtype="uint8")
        cv2.drawContours(mask, [c], -1, 255, -1)
 
        # apply the mask to the thresholded image, then
        # count the number of non-zero pixels in the
        # bubble area
        mask = cv2.bitwise_and(thresh, thresh, mask=mask)
        total = cv2.countNonZero(mask)
 
        # if the current total has a larger number of total
        # non-zero pixels, then we are examining the currently
        # bubbled-in answer
        if bubbled is None or total > bubbled[0]:
            bubbled = (total, j)
        
    # initialize the contour color and the index of the
    # *correct* answer
    color = (0, 0, 255)
    k = ANSWER_KEY[q]
 
    # check to see if the bubbled answer is correct
    if k == bubbled[1]:
        color = (0, 255, 0)
        correct += 1
 
    # draw the outline of the correct answer on the test
    cv2.drawContours(respuesta, [cnts[k]], -1, color, 3)
        

In [103]:
cv2.imshow("respuesta", respuesta)
cv2.waitKey(0)


-1

Given a row of bubbles, the next step is to determine which bubble is filled in.

We can accomplish this by using our thresh  image and counting the number of non-zero pixels (i.e., foreground pixels) in each bubble region:

Line 98 handles looping over each of the sorted bubbles in the row.

We then construct a mask for the current bubble on Line 101 and then count the number of non-zero pixels in the masked region (Lines 107 and 108). The more non-zero pixels we count, then the more foreground pixels there are, and therefore the bubble with the maximum non-zero count is the index of the bubble that the the test taker has bubbled in (Line 113 and 114).

Below I have included an example of creating and applying a mask to each bubble associated with a question:

In [83]:



# loop over the contours
output = paper.copy()
colors = [(220,20,60), (255,215,0), (34,139,34), (32,178,170), (128,0,0)]

i= 0

for question in questionCnts:
    
    color = colors[i//5]
    
    # compute the bounding box of the contour and then draw the
    # bounding box on both input images to represent where the two
    # images differ
    contours_poly = cv2.approxPolyDP(question, 3, True)
    
    center, radiu = cv2.minEnclosingCircle(contours_poly)
    
    
    cv2.circle(output, (int(center[0]), int(center[1])), int(radiu), color, 2)
    
    i += 1

cv2.imshow("questions", output)
cv2.waitKey(0)


24

Clearly, the bubble associated with “B” has the most thresholded pixels, and is therefore the bubble that the user has marked on their exam.

This next code block handles looking up the correct answer in the ANSWER_KEY , updating any relevant bookkeeper variables, and finally drawing the marked bubble on our image:

In [64]:
cv2.imshow("respuesta", respuesta)
cv2.waitKey(0)

-1

In [ ]:
# grab the test taker
score = (correct / 5.0) * 100
print("[INFO] score: {:.2f}%".format(score))
cv2.putText(respuesta, "{:.2f}%".format(score), (10, 30),
    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
cv2.imshow("Original", image)
cv2.imshow("Exam", respuesta)
cv2.waitKey(0)

[INFO] score: 80.00%


In [91]:
for j, c in enumerate(cnts):
    print(j, c)

(0, array([[[ 81, 270]],

       [[ 80, 271]],

       [[ 78, 271]],

       [[ 77, 272]],

       [[ 76, 272]],

       [[ 70, 278]],

       [[ 70, 279]],

       [[ 69, 280]],

       [[ 69, 281]],

       [[ 68, 282]],

       [[ 68, 291]],

       [[ 69, 292]],

       [[ 69, 294]],

       [[ 71, 296]],

       [[ 71, 297]],

       [[ 74, 300]],

       [[ 75, 300]],

       [[ 76, 301]],

       [[ 77, 301]],

       [[ 78, 302]],

       [[ 79, 302]],

       [[ 80, 303]],

       [[ 90, 303]],

       [[ 91, 302]],

       [[ 92, 302]],

       [[ 93, 301]],

       [[ 94, 301]],

       [[100, 295]],

       [[100, 294]],

       [[101, 293]],

       [[101, 291]],

       [[102, 290]],

       [[102, 282]],

       [[101, 281]],

       [[101, 280]],

       [[100, 279]],

       [[100, 278]],

       [[ 98, 276]],

       [[ 98, 275]],

       [[ 97, 274]],

       [[ 96, 274]],

       [[ 94, 272]],

       [[ 93, 272]],

       [[ 92, 271]],

       [[ 90, 271]],

      